In [6]:
import sys
import os
from typing import Tuple
import pandas

sys.path.append('/home/danillorp/Área de Trabalho/github/fema/src/')


import fema_semi
import fema_classifier
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
import matplotlib.pyplot as plt
from sklearn import preprocessing

DEBUG = False

In [7]:
ml_datasets = ['credit-g','satellite', 'Titanic','vehicle','blood-transfusion-service-center','one-hundred-plants-texture','steel-plates-fault','kc1','pc1','cmc']

#'credit-g','airlines','satellite',
#'Titanic','vehicle','adult','SpeedDating','blood-transfusion-service-center','mnist_784','one-hundred-plants-texture','steel-plates-fault',
#'kr-vs-kp','arrhythmia','bank-marketing','PhishingWebsites','kc1','pc1','cmc','mfeat-factors','KDDCup09_appetency'
print('The experiment will run for ',len(ml_datasets))        

The experiment will run for  10


In [8]:
def experiment(dataset_name:str, data_x:pandas.core.frame.DataFrame, data_y:pandas.core.series.Series, test_size:float, n_runs:int, DEBUG:bool=False):
    for i in range(n_runs):
        #split datasets
        train_x, test_x, train_y, test_y = train_test_split(data_x, data_y, test_size=test_size)
        uknw_x, test_x, uknw_y, test_y = train_test_split(test_x, test_y, test_size=test_size)
    
        if DEBUG:
            print('**********',dataset_name,'**********')
            print('train_size',len(train_x),'classes:',set(train_y))
            print('test_size',len(test_x),'classes:',set(test_y))
            print('uknw_size',len(uknw_x),'classes:',set(uknw_y))
    
        #reorganize the arrays to be an input of FEMa library
        train_y = train_y[:,np.newaxis]
        test_y = test_y[:,np.newaxis]
        uknw_y = uknw_y[:,np.newaxis]
    
        #Apply data scaling based on training data
        scaler = StandardScaler()
    
        train_x = scaler.fit_transform(train_x)
        test_x = scaler.transform(test_x)
        uknw_x = scaler.transform(uknw_x)
    
        #Run the model without the SemiSupervised
        model_fema_original = fema_classifier.FEMaClassifier(k=50,basis=fema_classifier.Basis.shepardBasis)
        model_fema_original.fit(train_x,train_y)
    
        pred, confidence_level = model_fema_original.predict(test_x,2)
    
        cm_fema_original = confusion_matrix(test_y,pred)
        bal_acc_original = balanced_accuracy_score(test_y, pred)
        acc_original = accuracy_score(test_y, pred)
    
        #print the accuracy of original solution
        if DEBUG:
            print('**********',dataset_name,'**********',i)
            print('********** original **********')
            print(cm_fema_original) 
            print(bal_acc_original, acc_original)
    
    
        #Run the Semi Supervised FEMa
        model_semi = fema_semi.FEMaSemiSupervisedClassifier(k=50,basis=fema_semi.Basis.shepardBasis)
        model_semi.fit(train_x,train_y,uknw_x,2)
        
        pred, confidence_level = model_semi.predict(test_x,2)
        
        cm_fema_semi = confusion_matrix(test_y,pred)
        bal_acc_semi = balanced_accuracy_score(test_y, pred)
        acc_semi = accuracy_score(test_y, pred)
        
        #print the accuracy of original solution
        if DEBUG:
            print('********** semi **********')
            print(cm_fema_semi) 
            print(bal_acc_semi)
            print(acc_semi)
    
        #Run the Semi Supervised FEMa with Play Probabilities
        """model_semi.playProbabilities()
        pred, confidence_level = model_semi.predict(test_x,2)
        
        cm_fema_semi = confusion_matrix(test_y,pred)
        bal_acc_semi = balanced_accuracy_score(test_y, pred)
        acc_semi = accuracy_score(test_y, pred)
        
        #print the accuracy of original solution
        if DEBUG:
            print('********** play **********')
            print(cm_fema_semi) 
            print(bal_acc_semi)
            print(acc_semi)
        """
        #Run the Semi Supervised FEMa with Round Probabilities
        model_semi.roundProbabilities()
        pred, confidence_level = model_semi.predict(test_x,2)
        
        cm_fema_semi = confusion_matrix(test_y,pred)
        bal_acc_semi = balanced_accuracy_score(test_y, pred)
        acc_semi = accuracy_score(test_y, pred)
        
        #print the accuracy of original solution
        if DEBUG:
            print('********** round **********')
            print(cm_fema_semi) 
            print(bal_acc_semi)
            print(acc_semi)

        
        
        return


In [9]:
for dataset in ml_datasets: 
    data = fetch_openml(dataset,version=1)
    X, y = data['data'], data['target']

    X = X.select_dtypes(include=np.number)

    le = preprocessing.LabelEncoder()
    le.fit(y)
    y = le.transform(y)

    if DEBUG:
        print(dataset, X.shape, y.shape, type(X), type(y))
        print(X.dtypes)
    
    experiment(dataset_name=dataset, data_x=X, data_y=y, test_size=0.50,n_runs=3,DEBUG=True)

********** credit-g **********
train_size 500 classes: {0, 1}
test_size 250 classes: {0, 1}
uknw_size 250 classes: {0, 1}


/home/danillorp/.local/lib/python3.10/site-packages/sklearn/datasets/_openml.py:1022: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


********** credit-g ********** 0
********** original **********
[[ 10  63]
 [ 11 166]]
0.537419704357248 0.704
250 2
********** semi **********
[[  7  66]
 [  7 170]]
0.528171194180017
0.708
********** round **********
[[  5  68]
 [  5 172]]
0.520122281557155
0.708


/home/danillorp/.local/lib/python3.10/site-packages/sklearn/datasets/_openml.py:1022: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


********** satellite **********
train_size 2550 classes: {0, 1}
test_size 1275 classes: {0, 1}
uknw_size 1275 classes: {0, 1}
********** satellite ********** 0
********** original **********
[[   5   15]
 [   0 1255]]
0.625 0.9882352941176471
1275 2
********** semi **********
[[   7   13]
 [   0 1255]]
0.675
0.9898039215686274
********** round **********
[[   8   12]
 [   0 1255]]
0.7
0.9905882352941177
********** Titanic **********
train_size 654 classes: {0, 1}
test_size 328 classes: {0, 1}
uknw_size 327 classes: {0, 1}


...

********** cmc ********** 0
********** original **********
[[146   2   3]
 [ 90   5   1]
 [115   0   7]]
0.35878259991073475 0.4281842818428184
368 3


/home/danillorp/Área de Trabalho/github/fema/src/fem_basis.py:33: RuntimeWarning: divide by zero encountered in divide
  dist = 1.0/(dist**z)
/home/danillorp/Área de Trabalho/github/fema/src/fem_basis.py:34: RuntimeWarning: invalid value encountered in divide
  weitghs = dist[mask]/sum(dist[mask]+0.0000000001)


********** semi **********
[[149   2   0]
 [ 96   0   0]
 [120   0   2]]
0.33438280317012264
0.4092140921409214


/home/danillorp/Área de Trabalho/github/fema/src/fem_basis.py:33: RuntimeWarning: divide by zero encountered in divide
  dist = 1.0/(dist**z)
/home/danillorp/Área de Trabalho/github/fema/src/fem_basis.py:34: RuntimeWarning: invalid value encountered in divide
  weitghs = dist[mask]/sum(dist[mask]+0.0000000001)


********** round **********
[[149   2   0]
 [ 95   0   1]
 [120   0   2]]
0.33438280317012264
0.4092140921409214
